### Generates the Records_df and from that computes and exports Frequency_df
### Frequency_df is the only one directly used by the Pathapp

In [1]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from sample_data import type_list,value_sign,day,adequacy_list,choices,last_week_date, last_month_date,results_list, genotype_list

## Generates the numer of tests processed in a given date

In [3]:
### this function defines how busy each day of the week is
def busy_days(n:int):
    ans=100
    if n==6:
        ans=0
    return ans
    
    
    

In [4]:
### this function defines how busy each month is
def busy_months(n:int):
    ans=1
    if n ==12:
        ans= 0.8
    if n in [1,2,6,7]:
        ans=1.2
    return ans    
    
    

In [5]:
def number_of_tests(fecha): ### This takes a date and returns how many tests occur
    ans=int (busy_days(fecha.weekday())*busy_months(fecha.month)*np.random.uniform(0.95,1.05))
    return ans


## Generate a new row given a data

In [32]:
def make_row(fecha):############ This takesa a date and returns an entry in the Records_df
    type_of_test=random.choices(type_list,weights=[3,4])[0]
    adequacy=random.choices(adequacy_list,weights=[5,2,1])[0]
    result=random.choices(results_list,weights=[4,1,1,1,1])[0]
    gen_type=random.choice(genotype_list)
    cytology=random.choices(value_sign,weights=[4,1])[0]+"cytology"
    hystology="Not prescribed"
    if cytology=="Positivecytology":
        hystology=random.choices(value_sign,weights=[92,8])[0]+"hystology"
    need_surgery=0
    if cytology=="Negativecytology":
        need_surgery=random.choices([0,1],weights=[87,13])[0]
    
    new_row = pd.DataFrame([{'day':fecha.date(), 'type':type_of_test,'adequacy':adequacy, 'result':result,'genotype':gen_type,'cytology':cytology,'hystology':hystology,"need_surgery":need_surgery}])
    return new_row

## Choose the date range 

In [33]:
starting_date = date.today() - timedelta(days = 366)
ending_date=date.today() +timedelta(days = 20)

## Generate Records_df

In [34]:
def generate_test_df():
    columns=['day', 'type', 'adequacy', 'result', 'genotype', 'cytology', 'hystology']
    records_df = pd.DataFrame(columns=columns)
    daterange = pd.date_range(starting_date, ending_date, freq='D')
    for item in daterange:
        N_of_test=number_of_tests(item)
        for patient in range(N_of_test):
            new_row=make_row(item)
            records_df = pd.concat([records_df, new_row], axis=0, ignore_index=True)
    return records_df

In [35]:
Records_df=generate_test_df()

In [36]:
Records_df.shape

(34938, 8)

In [37]:
Records_df[Records_df["cytology"]=="Positivecytology"].shape

(28081, 8)

In [41]:
Records_df["need_surgery"].sum()

925.0

In [24]:
Records_df[Records_df["need_surgery"]==0].shape

(27975, 8)

### This is the dataframe that the app takes

In [42]:
Records_df.to_csv("NewRecords.csv")

### GENERATE FREQUENCY DATA FRAME

In [18]:
def new_row_frequency(df,fecha):
    columnas= list(Records_df.columns)[1:]
    new_row_in={"day":fecha}
    df_fecha=df[df["day"]==fecha]
    new_row_in["All"]=df_fecha.shape[0]
    for columna in columnas:
        column_values=list(df_fecha[columna].unique())
        for tipo in column_values:
            temp_df=df_fecha[df_fecha[columna]==tipo]
            new_row_in[tipo]=temp_df.shape[0]
    types=list(df_fecha["type"].unique())
    adequacies=list(df_fecha["adequacy"].unique())
    cytologies=list(df_fecha["cytology"].unique())
    hystologies=list(df_fecha["hystology"].unique())
    for tipo in types:
        for ad in adequacies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["adequacy"]==ad)]
            new_row_in[tipo+ad]=temp_df.shape[0]
        for cyt in cytologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["cytology"]==cyt)]
            new_row_in[tipo+cyt]=temp_df.shape[0]
        for hyst in hystologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["hystology"]==hyst)]
            new_row_in[tipo+hyst]=temp_df.shape[0]


        # new_row_in["weekday"]=week_day(fecha.weekday())
    
    return pd.DataFrame([new_row_in])

In [19]:
def make_frequency_df(df):
    fechas=df["day"].unique()
    freq_df=pd.DataFrame()
    for fecha in fechas:
        freq_df=pd.concat([freq_df,new_row_frequency(df,fecha)],axis=0, ignore_index=True)
    
    return freq_df

In [20]:
Frequency_df=make_frequency_df(Records_df)

In [17]:
#Frequency_df.to_csv('Frequency.csv')#

In [8]:
from be.controllers.bar_chart import make_bar

In [9]:
import pandas as pd

In [2]:
df=pd.read_excel("data/USCAP.xlsx")

In [11]:
df.head()

,YEAR,SURGICAL_NUMBER,ACCESS_DATE,SIGN_DATE,SEX,AGE,DIAGNOSIS,SITE,RIGHT,LEFT,ISTHMUS,LATERALITY,Bethesda Cathegory,CYTOPATHOLOGIST,CYTOTECHNOLOGIST,SIZE,MOLECULAR,ADEQUACY,RESULT,ROM,GENE MUTATED,MUTATION,VAF,FUSIONS,CNV,GEP,PTH,C-CELLS
0,2021,DEMO-21-845,2021-02-11,2021-02-16,Female,29,"Thyroid, Left, Fine Needle Aspiration; Smears ...",Thyroid,0,1,0,Left,3,9.0,MID,1.46,THYROSEQ,ADEQUATE,NEGATIVE,3,TSHR,"L512R,c.1535T>G",6,0,0.0,SLC5A5,0.0,0.0
1,2021,DEMO-21-8458,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Anterior, Fine Needle Aspiratio...",Thyroid,1,0,0,RightAnterior,6,2.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,DEMO-21-8459,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Posterior, Fine Needle Aspirati...",Thyroid,1,0,0,RightPosterior,6,2.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,DEMO-21-847,2021-02-11,2021-02-16,Female,48,"Thyroid, Right Middle, Fine Needle Aspiration;...",Thyroid,1,0,0,RightMiddle,2,9.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,DEMO-21-8476,2021-12-16,2021-12-20,Female,64,"Thyroid, Right, Fine Needle Aspiration; Smears...",Thyroid,1,0,0,Right,2,2.0,MJH,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
labels=sorted(list(df["Bethesda Cathegory"].unique()))

In [16]:
values=values=[len(df[df["Bethesda Cathegory"]==label])for label in labels]

In [17]:
bar_data=pd.DataFrame()
bar_data["values"]=values
bar_data["labels"]=labels
bar_data["labels"]=bar_data["labels"].astype(str)

In [18]:
make_bar(bar_data,labels,values)

In [20]:
df.columns

Index(['YEAR', 'SURGICAL_NUMBER', 'ACCESS_DATE', 'SIGN_DATE', 'SEX', 'AGE',
       'DIAGNOSIS', 'SITE', 'RIGHT', 'LEFT', 'ISTHMUS', 'LATERALITY',
       'Bethesda Cathegory', 'CYTOPATHOLOGIST', 'CYTOTECHNOLOGIST', 'SIZE',
       'MOLECULAR ', 'ADEQUACY', 'RESULT', 'ROM', 'GENE MUTATED', 'MUTATION',
       'VAF', 'FUSIONS', ' CNV', 'GEP', 'PTH', ' C-CELLS'],
      dtype='object')

In [21]:
df.head()

,YEAR,SURGICAL_NUMBER,ACCESS_DATE,SIGN_DATE,SEX,AGE,DIAGNOSIS,SITE,RIGHT,LEFT,ISTHMUS,LATERALITY,Bethesda Cathegory,CYTOPATHOLOGIST,CYTOTECHNOLOGIST,SIZE,MOLECULAR,ADEQUACY,RESULT,ROM,GENE MUTATED,MUTATION,VAF,FUSIONS,CNV,GEP,PTH,C-CELLS
0,2021,DEMO-21-845,2021-02-11,2021-02-16,Female,29,"Thyroid, Left, Fine Needle Aspiration; Smears ...",Thyroid,0,1,0,Left,3,9.0,MID,1.46,THYROSEQ,ADEQUATE,NEGATIVE,3,TSHR,"L512R,c.1535T>G",6,0,0.0,SLC5A5,0.0,0.0
1,2021,DEMO-21-8458,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Anterior, Fine Needle Aspiratio...",Thyroid,1,0,0,RightAnterior,6,2.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,DEMO-21-8459,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Posterior, Fine Needle Aspirati...",Thyroid,1,0,0,RightPosterior,6,2.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,DEMO-21-847,2021-02-11,2021-02-16,Female,48,"Thyroid, Right Middle, Fine Needle Aspiration;...",Thyroid,1,0,0,RightMiddle,2,9.0,MID,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,DEMO-21-8476,2021-12-16,2021-12-20,Female,64,"Thyroid, Right, Fine Needle Aspiration; Smears...",Thyroid,1,0,0,Right,2,2.0,MJH,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df["GENE MUTATED"].unique()

array(['TSHR', nan, 'NRAS', 0, 'EZH1', 'GNAS', 'HRAS', 'BRAF',
       'TERT,PIK3CA,PTEN'], dtype=object)

In [23]:
df["ROM"].unique()

array([3, nan, 50, 70, 99, 40, 0, '?'], dtype=object)

In [24]:
df["MUTATION"].unique()

array(['L512R,c.1535T>G', nan, 'Q61R', 0, 'Y642F', 'Q227H', 'Q61K',
       'D633H', 'Q571R', 'V600E', 'SLC5A5', 'M453T',
       'H1047R,D77Afs*22,C228T', 'S281N'], dtype=object)

In [3]:
from be.controllers.roman import make_roman

In [4]:
from be.controllers.bar_chart import make_bar

In [5]:
from be.controllers.pie_chart import make_pie

In [6]:
labels_bar=list(df["ROM"].unique())
values_bar=[len(df[df["ROM"]==label])for label in labels_bar]
bar_data=pd.DataFrame()
bar_data["Count"]=values_bar
bar_data["Category"]=labels_bar
# bar_data["labels"]=bar_data["labels"].astype(str)
second_graph=make_pie(labels_bar,values_bar,"Bethesda category counts")

In [7]:
second_graph.show()

In [10]:
labels_bar

[3, nan, 50, 70, 99, 40, 0, '?']

In [8]:
values_bar

[27, 0, 2, 5, 1, 1, 2, 1]

In [48]:
df["ROM"].value_counts()

3     27
70     5
50     2
0      2
99     1
40     1
?      1
Name: ROM, dtype: int64

In [13]:
df["GENE MUTATED"].value_counts()

0                   23
TSHR                 5
NRAS                 3
HRAS                 3
EZH1                 2
GNAS                 1
BRAF                 1
TERT,PIK3CA,PTEN     1
Name: GENE MUTATED, dtype: int64

In [14]:
cosas=[2,4,5,6]
otras= [x for x in cosas if x>2]

In [16]:
otras= [x for x in cosas if x not in [6]]

In [17]:
otras

[2, 4, 5]

In [21]:
df["MOLECULAR "].value_counts()

NO             152
THYROSEQ        39
BUSCAR EPIC      1
CANCELLED        1
Name: MOLECULAR , dtype: int64

In [23]:
df[df["MOLECULAR "]=="THYROSEQ"].shape[0]

39

In [24]:
df.columns

Index(['YEAR', 'SURGICAL_NUMBER', 'ACCESS_DATE', 'SIGN_DATE', 'SEX', 'AGE',
       'DIAGNOSIS', 'SITE', 'RIGHT', 'LEFT', 'ISTHMUS', 'LATERALITY',
       'Bethesda Cathegory', 'CYTOPATHOLOGIST', 'CYTOTECHNOLOGIST', 'SIZE',
       'MOLECULAR ', 'ADEQUACY', 'RESULT', 'ROM', 'GENE MUTATED', 'MUTATION',
       'VAF', 'FUSIONS', ' CNV', 'GEP', 'PTH', ' C-CELLS'],
      dtype='object')

In [25]:
df["ROM"].value_counts

<bound method IndexOpsMixin.value_counts of 0        3
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
190    NaN
191    NaN
192    NaN
193    NaN
194      3
Name: ROM, Length: 195, dtype: object>

In [34]:
values_bar=[len(df[df["ROM"]==label])for label in labels_bar]

In [36]:
sorted(values_bar)

[0, 1, 1, 1, 2, 2, 5, 27]

In [37]:
labels_bar=list(df["ROM"].unique())
labels_bar=[x for x in labels_bar if str(x) not in ["?","nan"]]

In [40]:
type(labels_bar[0])

int

In [41]:
df.columns

Index(['YEAR', 'SURGICAL_NUMBER', 'ACCESS_DATE', 'SIGN_DATE', 'SEX', 'AGE',
       'DIAGNOSIS', 'SITE', 'RIGHT', 'LEFT', 'ISTHMUS', 'LATERALITY',
       'Bethesda Cathegory', 'CYTOPATHOLOGIST', 'CYTOTECHNOLOGIST', 'SIZE',
       'MOLECULAR ', 'ADEQUACY', 'RESULT', 'ROM', 'GENE MUTATED', 'MUTATION',
       'VAF', 'FUSIONS', ' CNV', 'GEP', 'PTH', ' C-CELLS'],
      dtype='object')

In [43]:
df["GENE MUTATED"].value_counts()

0                   23
TSHR                 5
NRAS                 3
HRAS                 3
EZH1                 2
GNAS                 1
BRAF                 1
TERT,PIK3CA,PTEN     1
Name: GENE MUTATED, dtype: int64

In [47]:
df.head(10)

,YEAR,SURGICAL_NUMBER,ACCESS_DATE,SIGN_DATE,SEX,AGE,DIAGNOSIS,SITE,RIGHT,LEFT,...,RESULT,ROM,GENE MUTATED,MUTATION,VAF,FUSIONS,CNV,GEP,PTH,C-CELLS
0,2021,DEMO-21-845,2021-02-11,2021-02-16,Female,29,"Thyroid, Left, Fine Needle Aspiration; Smears ...",Thyroid,0,1,...,NEGATIVE,3,TSHR,"L512R,c.1535T>G",6,0,0.0,SLC5A5,0.0,0.0
1,2021,DEMO-21-8458,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Anterior, Fine Needle Aspiratio...",Thyroid,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,DEMO-21-8459,2021-12-15,2021-12-20,Female,59,"Thyroid, Right Posterior, Fine Needle Aspirati...",Thyroid,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,DEMO-21-847,2021-02-11,2021-02-16,Female,48,"Thyroid, Right Middle, Fine Needle Aspiration;...",Thyroid,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,DEMO-21-8476,2021-12-16,2021-12-20,Female,64,"Thyroid, Right, Fine Needle Aspiration; Smears...",Thyroid,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021,DEMO-21-8477,2021-12-16,2021-12-20,Female,28,"Thyroid, Left, Fine Needle Aspiration; Smears ...",Thyroid,0,1,...,POSITIVE,50,NRAS,Q61R,33,0,0.0,0,0.0,0.0
6,2021,DEMO-21-8478,2021-12-16,2021-12-20,Female,63,"Thyroid, Right Inferior, Fine Needle Aspiratio...",Thyroid,1,0,...,POSITIVE,70,NRAS,Q61R,13,0,0.0,POSITIVE,0.0,0.0
7,2021,DEMO-21-8489,2021-12-16,2021-12-20,Female,54,"Thyroid, Left Middle, Fine Needle Aspiration; ...",Thyroid,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021,DEMO-21-8490,2021-12-16,2021-12-20,Female,54,"Thyroid, Right Middle, Fine Needle Aspiration;...",Thyroid,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021,DEMO-21-8491,2021-12-16,2021-12-20,Female,80,"Thyroid, Left Inferior, Fine Needle Aspiration...",Thyroid,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NameError: name 'pathologists' is not defined

In [1]:
from be.controllers.default_times_ranges import df, last_day,first_day,years,pathologists

['Pathologist 2',
 'Pathologist 5',
 'Pathologist 7',
 'Pathologist 8',
 'Pathologist 9',
 'Pathologist 12']

In [5]:
import pandas as pd

['Pathologist 2',
 'Pathologist 5',
 'Pathologist 7',
 'Pathologist 8',
 'Pathologist 9',
 'Pathologist 12']

In [28]:
def make_gap_minder_data(dataframe):
    gap_minder_data=pd.DataFrame()
    for year in years:
        for pathologist in pathologists:
            new_row=dict()
            size=dataframe[(dataframe["CYTOPATHOLOGIST"]==pathologist)&(dataframe["YEAR"]==year)].shape[0]
            
            calls_CatIII=dataframe[(dataframe["CYTOPATHOLOGIST"]==pathologist)&(dataframe["YEAR"]
            ==year)&(dataframe["Bethesda Cathegory"]==3)].shape[0]
            if calls_CatIII>0:
                call_rate_CatIII=calls_CatIII/size
                
                positive_rate_CatIII=dataframe[(dataframe["CYTOPATHOLOGIST"]==pathologist)&(dataframe["YEAR"]
                ==year)&(dataframe["Bethesda Cathegory"]==3)&(dataframe["RESULT"]=="POSITIVE")].shape[0]/calls_CatIII
                new_row["Pathologist"]=[pathologist]
                new_row["Year"]=[year]
                new_row["Case Count"]=[size]
                new_row["Call rate category III"]=[call_rate_CatIII]
                new_row["Positive rate category III"]=[positive_rate_CatIII]
                new_row=pd.DataFrame.from_dict(new_row)
                gap_minder_data=pd.concat([gap_minder_data,new_row])
    for year in years:
        new_row=dict()
        size=dataframe[(dataframe["YEAR"]==year)].shape[0]
        
        calls_CatIII=dataframe[(dataframe["YEAR"]
        ==year)&(dataframe["Bethesda Cathegory"]==3)].shape[0]
        if calls_CatIII>0:
            call_rate_CatIII=calls_CatIII/size
            
            positive_rate_CatIII=dataframe[(dataframe["YEAR"]
            ==year)&(dataframe["Bethesda Cathegory"]==3)&(dataframe["RESULT"]=="POSITIVE")].shape[0]/calls_CatIII
            new_row["Pathologist"]=["All pathologists"]
            new_row["Year"]=[year]
            new_row["Case Count"]=[size]
            new_row["Call rate category III"]=[call_rate_CatIII]
            new_row["Positive rate category III"]=[positive_rate_CatIII]
            new_row=pd.DataFrame.from_dict(new_row)
            gap_minder_data=pd.concat([gap_minder_data,new_row])
    return gap_minder_data

In [32]:
make_gap_minder_data(df)

,Pathologist,Year,Case Count,Call rate category III,Positive rate category III
0,Pathologist 2,2018,491,0.128310,0.222222
0,Pathologist 5,2018,232,0.379310,0.147727
0,Pathologist 7,2018,4,0.500000,0.000000
0,Pathologist 8,2018,480,0.139583,0.298507
0,Pathologist 9,2018,85,0.129412,0.272727
0,Pathologist 2,2019,358,0.094972,0.323529
0,Pathologist 5,2019,425,0.230588,0.214286
0,Pathologist 7,2019,229,0.065502,0.400000
0,Pathologist 8,2019,358,0.131285,0.361702
0,Pathologist 9,2019,114,0.157895,0.388889


In [34]:
gap_minder_data=make_gap_minder_data(df)

In [35]:
path_time_df=dict()
for path in pathologists:
    path_time_df[path]=pd.DataFrame()
    path_time_df[path]["x"]=list(gap_minder_data[gap_minder_data["Pathologist"]==path]["Year"].unique())
    path_time_df[path]["y"]=list(gap_minder_data[gap_minder_data["Pathologist"]==path]["Call rate category III"])

    

In [36]:
from be.controllers.scater_graph_bethesda_time import make_scatter_graph_time_bethesda

In [37]:
make_scatter_graph_time_bethesda(path_time_df)

In [44]:
a

'Pathologist 2'

In [42]:
a=list(path_time_df.keys())[0]

In [46]:
path_time_df[a]

,x,y
0,2018,0.128310
1,2019,0.094972
2,2020,0.101010
3,2021,0.141593
4,2022,0.121622


In [47]:
import plotly.express as px
# df = px.data.iris() # iris is a pandas DataFrame
fig = px.scatter(path_time_df[a], x="x", y="x")
fig.show()

In [51]:
import pandas as pd
import plotly.express as px

iris = px.data.iris()
fig = px.scatter()
df = pd.DataFrame({
    'x':[1,2,3,4],
    'y':[5,6,7,8],})
fig2 = px.bar(df, x="x", y="y")
fig.add_trace(fig2.data[0])
fig.show()

In [56]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.line(df, x='year', y='lifeExp',color="year" ,markers=True)
fig.show()

In [3]:
from be.controllers.default_times_ranges import df

In [4]:
import pandas as pd

In [7]:
3+4

7

In [8]:
from be.controllers.

3